<a href="https://colab.research.google.com/github/V4lciJr/criacao-de-base-de-dados-e-treinamento-da-rede-YOLO/blob/main/Cria%C3%A7%C3%A3o_de_base_de_dados_e_treinamento_da_rede_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🧠 OBJETIVO

➡️ Fazer fine-tuning do YOLOv3 pré-treinado no COCO (80 classes originais) adicionando 2 classes novas personalizadas ao modelo.

In [ ]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make

In [ ]:
# Cria pasta de dados
%cd /content
!mkdir -p data && cd data

# Baixa imagens e anotações COCO (val2017 como exemplo)
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip -q val2017.zip -d data/

!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q annotations_trainval2017.zip -d data/

In [24]:
import json
import os
from pathlib import Path
from tqdm import tqdm

json_path = "/content/data/annotations/instances_val2017.json"
images_dir = "/content/data/val2017"
output_dir = "/content/data/val2017/labels"

os.makedirs(output_dir, exist_ok=True)

# COCO classes
with open(json_path) as f:
    coco = json.load(f)

# Mapeia categoria para nome e índice
categories = {cat['id']: cat['name'] for cat in coco['categories']}
category_id_map = {cat['id']: i for i, cat in enumerate(coco['categories'])}

# Agrupa anotações por imagem
annotations_by_image = {}
for ann in coco['annotations']:
    image_id = ann['image_id']
    annotations_by_image.setdefault(image_id, []).append(ann)

# Cria arquivos de label no formato YOLO
for img in tqdm(coco['images']):
    img_id = img['id']
    file_name = img['file_name']
    width = img['width']
    height = img['height']
    label_path = os.path.join(output_dir, Path(file_name).stem + '.txt')

    with open(label_path, 'w') as f:
        for ann in annotations_by_image.get(img_id, []):
            cat_id = ann['category_id']
            x, y, w, h = ann['bbox']
            x_center = (x + w / 2) / width
            y_center = (y + h / 2) / height
            w /= width
            h /= height
            class_id = category_id_map[cat_id]
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")


100%|██████████| 5000/5000 [00:00<00:00, 7773.94it/s]


In [25]:
# Novas classes adicionadas "CAPACETE", "MASCARA"
classes = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
    "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop",
    "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush",
    "capacete", "mascara"
]

os.makedirs("cfg", exist_ok=True)

with open("cfg/obj.names", "w") as f:
    for cls in classes:
        f.write(cls + "\n")

In [27]:
!find /content/data/val2017 -name "*.jpg" > /content/darknet/data/train.txt

In [18]:
with open("/content/darknet/data/obj.data", "w") as f:
    f.write(f"classes = {len(classes)}\n")
    f.write("train = data/train.txt\n")
    f.write("names = data/obj.names\n")
    f.write("backup = backup/\n")

In [28]:
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg -O /content/darknet/cfg/yolov3.cfg


--2025-07-20 17:42:23--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘/content/darknet/cfg/yolov3.cfg’

/content/darknet/cf 100%[===================>]   8.15K  --.-KB/s    in 0s      

2025-07-20 17:42:23 (81.2 MB/s) - ‘/content/darknet/cfg/yolov3.cfg’ saved [8342/8342]



In [29]:
# Altera cfg para yolov3-custom.cfg com 82 classes
cfg_path = "/content/darknet/cfg/yolov3.cfg"
new_cfg_path = "/content/darknet/cfg/yolov3-custom.cfg"

with open(cfg_path) as f:
    lines = f.readlines()

with open(new_cfg_path, "w") as f:
    for i in range(len(lines)):
        if "filters=" in lines[i]:
            # A camada [convolutional] antes da [yolo] deve ser ajustada
            f.write("filters=255\n" if "yolo" not in lines[i+1].lower() else "filters=255\n")
        elif "classes=" in lines[i]:
            f.write("classes=82\n")
        else:
            f.write(lines[i])


In [35]:
!./darknet/darknet detector train darknet/data/obj.data cfg/yolov3-custom.cfg yolov3.weights -dont_show -map


In [37]:
!wget https://pjreddie.com/media/files/darknet53.conv.74
!./darknet/darknet detector train darknet/data/obj.data cfg/yolov3-custom.cfg darknet53.conv.74 -dont_show -map


--2025-07-20 17:44:52--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 172.67.185.199, 104.21.88.156, 2606:4700:3037::6815:589c, ...
Connecting to pjreddie.com (pjreddie.com)|172.67.185.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘darknet53.conv.74.4’

darknet53.conv.74.4     [ <=>                ]   8.88K  --.-KB/s    in 0s      

2025-07-20 17:44:53 (81.4 MB/s) - ‘darknet53.conv.74.4’ saved [9093]



In [40]:
!./darknet/darknet detector test data/obj.data cfg/yolov3-custom.cfg backup/yolov3-custom_final.weights -thresh 0.3 -dont_show -ext_output -out result.json data/val2017/000000000139.jpg